In [124]:
import pymorton as pm
from random import randrange
import numpy as np
import zCurve as zC
from itertools import permutations

def coordinate_permutations(coord):
    """
    Given a coordinate (tuple or list) of length D,
    return all D! permutations as tuples.
    """
    return list(permutations(coord))


In [4]:
from random import randrange

bit_size = 16
max_val = 2**bit_size - 1
no_samples = 10**6
data_points = [(randrange(0, max_val), randrange(0, max_val), randrange(0, max_val)) for i in range(no_samples)]

In [13]:
import zCurve as z 
morton_codes = z.par_interlace(data_points, dims=3, bits_per_dim=16)
#data_points == z.par_deinterlace(morton_codes, dims=3)
len(morton_codes)


1000000

In [ ]:
N=2

arr=np.linspace(1,48,48).reshape((3,4,4)).astype(np.uint8)
arr=np.ones((N,N,N)).astype(np.uint8)
dims = arr.ndim
Nx, Ny, Nz = arr.shape
bits_per_dim=int(np.log2(max(Nx, Ny, Nz)))
Npermutations = np.math.factorial(dims)
#print(arr)

# choose uint type based on bits_per_dim
if bits_per_dim * dims <= 8:
    dtype = np.uint8
elif bits_per_dim * dims <= 16:
    dtype = np.uint16
elif bits_per_dim * dims <= 32:
    dtype = np.uint32
else:
    dtype = np.uint64

print(dtype)

# Compute Morton codes for each voxel
codes1 = np.empty((Nx, Ny, Nz, Npermutations), dtype=dtype)
codes = np.empty((Nx, Ny, Nz, Npermutations), dtype=dtype)
for x in range(Nx):
    for y in range(Ny):
        for z in range(Nz):
          #  codes1[x, y, z] = pm.interleave3(x, y, z)
            permutation_list = coordinate_permutations((x, y, z))
          #  zperms = zC.par_interlace(permutation_list, dims=3, bits_per_dim=bits_per_dim)
            permutation_list = [(x,y,z), (x,z,y), (y,x,z), (y,z,x), (z,x,y), (z,y,x)]
            codes[x, y, z, 0] = zC.interlace(x, y, z, dims=3, bits_per_dim=bits_per_dim)
         #   codes[x, y, z, 1] = zC.interlace(x, z, y, dims=3, bits_per_dim=bits_per_dim)
            print(type(x), type(y), type(z), type(permutation_list[0][0]))
            codes[x, y, z, :] = zC.par_interlace(permutation_list, dims=3, bits_per_dim=bits_per_dim)
            print(codes[x, y, z, :])
          #  print(codes[x, y, z, 0], codes[x, y, z, 1])

#print(codes)
# Flatten both arrays and sort by Morton code
flat_arr = arr.ravel()
flat_codes = [codes[..., i].ravel() for i in range(Npermutations)]
orders = [np.argsort(flat_codes[i]) for i in range(Npermutations)]
zorders = [flat_arr[orders[i]] for i in range(Npermutations)]
#order = np.argsort(flat_codes)
#order1 = np.argsort(codes1.ravel())
#z_ordered = flat_arr[order]
#print(order), print(flat_arr[order1 ]), print(np.max(order)), print("O", len(order), np.unique(order).shape)
orders

In [241]:
import numpy as np
import itertools
import zCurve as zC  # your Morton/Z-order library

def all_permutation_codes_listmode(Nx, Ny, Nz, bits_per_dim, dtype=np.uint64):
    """
    Compute Morton codes for every (x, y, z) voxel for all 3! coordinate permutations.
    Uses zC.par_interlace(), which expects a list of tuples.
    """
    D = 3
    perms = list(itertools.permutations(range(D)))
    Nperm = len(perms)

    # Make coordinate grid (Nx, Ny, Nz, 3)
    x, y, z = np.meshgrid(
        np.arange(Nx, dtype=dtype),
        np.arange(Ny, dtype=dtype),
        np.arange(Nz, dtype=dtype),
        indexing="ij"
    )
    coords = np.stack((x, y, z), axis=-1).reshape(-1, D)  # (Npoints, 3)

    # Convert to list of tuples once
    coord_list = [tuple(c) for c in coords]  # e.g. [(0,0,0), (0,0,1), ...]

    # Preallocate output
    codes_flat = np.empty((len(coord_list), Nperm), dtype=dtype)

    # Process all 6 permutations — each with one call to par_interlace
    for i, p in enumerate(perms):
        # reorder coordinates according to permutation
        permuted_list = [(int(c[p[0]]), int(c[p[1]]), int(c[p[2]])) for c in coord_list]

        zvals=zC.par_interlace(permuted_list, dims=3, bits_per_dim=bits_per_dim)
        codes_flat[:, i] = zC.par_interlace(permuted_list, dims=3, bits_per_dim=bits_per_dim)

    orders = [np.argsort(codes_flat[:, i]).astype(dtype) for i in range(Nperm)]
    return orders

Nx, Ny, Nz = 3,4,4
bits_per_dim = max(Nx, Ny, Nz).bit_length()+1
print(bits_per_dim)
codes = all_permutation_codes_listmode(Nx, Ny, Nz, bits_per_dim=bits_per_dim**3, dtype=np.uint8)


4


In [242]:
codes

[array([ 0, 16,  4, 20,  1, 17,  5, 21, 32, 36, 33, 37,  8, 24, 12, 28,  9,
        25, 13, 29, 40, 44, 41, 45,  2, 18,  6, 22,  3, 19,  7, 23, 34, 38,
        35, 39, 10, 26, 14, 30, 11, 27, 15, 31, 42, 46, 43, 47],
       dtype=uint8),
 array([ 0, 16,  1, 17,  4, 20,  5, 21, 32, 33, 36, 37,  2, 18,  3, 19,  6,
        22,  7, 23, 34, 35, 38, 39,  8, 24,  9, 25, 12, 28, 13, 29, 40, 41,
        44, 45, 10, 26, 11, 27, 14, 30, 15, 31, 42, 43, 46, 47],
       dtype=uint8),
 array([ 0,  4, 16, 20,  1,  5, 17, 21,  8, 12, 24, 28,  9, 13, 25, 29, 32,
        36, 33, 37, 40, 44, 41, 45,  2,  6, 18, 22,  3,  7, 19, 23, 10, 14,
        26, 30, 11, 15, 27, 31, 34, 38, 35, 39, 42, 46, 43, 47],
       dtype=uint8),
 array([ 0,  4,  1,  5, 16, 20, 17, 21,  8, 12,  9, 13, 24, 28, 25, 29,  2,
         6,  3,  7, 18, 22, 19, 23, 10, 14, 11, 15, 26, 30, 27, 31, 32, 36,
        33, 37, 40, 44, 41, 45, 34, 38, 35, 39, 42, 46, 43, 47],
       dtype=uint8),
 array([ 0,  1, 16, 17,  4,  5, 20, 21,  2,  3, 